# Lets build out first Deep Neural Network

All dependencies for this notebook is listed in the requirements.txt file. One parent above the nbs directory. This list will keep changing as we add to it so be sure to rerun this line after every git pull

In [ ]:
!pip install -r ../requirements.txt

Lets declare our imports

In [ ]:
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import math

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
 ! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets list

/bin/sh: kaggle: command not found


In [14]:
! kaggle datasets download -d ronitf/heart-disease-uci

/bin/sh: kaggle: command not found


In [137]:
class MyFirstNeuralNetwork(torch.nn.Module):
    def __init__(self, in_size=2, out_size=2, hidden_size=3):

        super(MyFirstNeuralNetwork, self).__init__()

        # Set the dimensionality of the network
        self.input_size = in_size
        self.output_size = out_size
        self.hidden_size = hidden_size

        # Initialize our weights
        self._init_weights()

    '''
    Initialize the weights
    '''
    def _init_weights(self):
        # Create an input tensor of shape (in_size, hidden_size)
        self.W_Input = torch.randn(self.input_size, self.hidden_size)
        # Create an output tensor of shape (3, 1)
        self.W_Output = torch.randn(self.hidden_size, self.output_size)
        
        print(self.W_Input.shape, self.W_Output.shape)

    '''
    Create the forward pass
    '''
    def forward(self, inputs):
        # Lets get the element wise dot product
        self.z = torch.matmul(inputs, self.W_Input)
        # We call the activation
        self.state = self._activation(self.z)
        # Pass it through the hidden layer
        self.z_hidden = torch.matmul(self.state, self.W_Output)
        # Finally activate the output
        output = self._activation(self.z_hidden)
        # Return the output
        print(self.z.shape, self.state.shape, self.z_hidden.shape, output.shape)
        
        return output

    '''
    Backpropagation algorithm implemented
    '''
    def backward(self, inputs, labels, output):
        # What is the error in output
        self.loss = labels - output
        # What is the delta loss based on the derivative
        self.loss_delta = self.loss * self._derivative(output)
        # Get the loss for the existing output weight
        print(self.loss_delta.shape, torch.t(self.W_Output).shape)
        
        self.z_loss = torch.matmul(self.loss_delta, torch.t(self.W_Output))
        # Compute the delta like before
        self.z_loss_delta = self.z_loss * self._derivative(self.state)
        # Finally propogate this to our existing weight tensors to update
        # the gradient loss
        self.W_Input += torch.matmul(torch.t(inputs), self.z_loss_delta)
        self.W_Output += torch.matmul(torch.t(self.state), self.loss_delta)

    '''
    Here we train the network
    '''
    def train(self, inputs, labels):
        # First we do the foward pass
        outputs = self.forward(inputs)
        # Then we do the backwards pass
        self.backward(inputs, labels, outputs)

    '''
    Here we perform inference
    '''
    def predict(self, inputs):
        pass

    '''
    Here we save the model
    '''
    def save(self, out_path):
        self.save(out_path)
    
    '''
    Our non-linear activation function
    '''
    def _activation(self, s):
        # Lets use sigmoid
        return 1 / (1 * torch.exp(-s))

    '''
    Our derivative function used for backpropagation
    Usually the sigmoid prime
    '''
    def _derivative(self, s):
        # derivative of sigmoid
        return s * (1 - s)

In [138]:
import pandas as pd
df = pd.read_csv('../data/heart.csv')

In [139]:
df.head(20)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [140]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler((-1, 1))

Lets split out dataset between inputs and target

In [141]:
df.shape
y = df['target']
X = df.drop('target', axis=1)

Lets create a test and train split

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Here we transform features by scaling each feature to a given range.
This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

In [149]:
X_train = sc.fit_transform(X_train)

In [150]:
X_train = torch.tensor(X_train).float()

In [194]:
y_train = torch.tensor((np.array(y_train.values,))

In [195]:
print(X_train.shape, y_train.shape)

(227, 13) torch.Size([1, 227])


Now we instantiate our neural network

In [190]:
nn = MyFirstNeuralNetwork(in_size=X_train.shape[1], out_size=1)

torch.Size([13, 3]) torch.Size([3, 1])


We train our neural network with 1000 epochs (training loops) and we measure the loss

In [176]:
for i in range(10):
    outputs = nn(X_train)
    loss = torch.mean((labels - outputs)**2).detach().item()
    print("Loss: {}".format(loss))
    nn.train(X_train, labels)

torch.Size([227, 3]) torch.Size([227, 3]) torch.Size([227, 2]) torch.Size([227, 2])
tensor([[5.2954e-02, 3.9217e-04],
        [1.7577e+00, 7.4750e-01],
        [9.1245e-01, 4.6959e-01],
        [2.8975e+00, 4.1316e-01],
        [1.6487e+00, 9.1777e-01],
        [1.4421e+00, 7.8686e-01],
        [1.7682e+00, 6.0586e-01],
        [3.3079e-01, 3.9138e-02],
        [1.5003e-01, 3.0300e-05],
        [1.8591e-03, 1.1757e-07],
        [1.3428e+00, 4.1034e-01],
        [3.3279e-01, 5.0488e-02],
        [7.0771e-27, 0.0000e+00],
        [7.9000e-01, 3.5713e-01],
        [2.6576e+00, 1.5059e-01],
        [6.9164e+00, 1.1266e+00],
        [7.3010e-01, 2.9211e-01],
        [1.2784e+00, 8.7293e-01],
        [9.0858e-01, 5.9425e-01],
        [1.1721e+00, 8.1714e-01],
        [7.4320e-01, 9.3856e-02],
        [7.8127e-04, 2.3504e-08],
        [1.3018e+00, 8.8582e-01],
        [1.0650e+00, 1.8940e-01],
        [1.2064e+00, 6.8964e-01],
        [1.1241e+00, 7.8098e-01],
        [1.5326e+00, 7.9939e-01]

RuntimeError: The size of tensor a (227) must match the size of tensor b (2) at non-singleton dimension 1

# Excercises

1. Try to initialize the weights with something better. Hint (Xavier Initialization)
2. Add a bias to the forward pass. Recall the affine transform is (inputs . weights) + bias
3. We are missing a learning rate to the backwards pass. See if you can add that in

# How would we rewrite this code using PyTorch built-in methods

PyTorch gives us most of this functionality out of the box. First we can flag all Tensors to use Autograd. You can read more about autograd here: https://pytorch.org/docs/stable/autograd.html

In [191]:
labelz= torch.tensor(([1],[0],[0],[1]), dtype=torch.float)
inputs = torch.tensor(([20, 90],[10, 20],[30, 40],[20, 50]), dtype=torch.float)

In [192]:
labelz.shape

torch.Size([4, 1])